Client code for Dropbox's API

In [1]:
import webbrowser
import http.server
import socketserver
from urllib import request
import urllib
import http.server
import socket
import re
import time
import json

In [2]:
f = open("app_info.txt", "r") ## Contains private info about application
APP_KEY = f.readline().rstrip().replace("app key ", "")
APP_SECRET = f.readline().rstrip().replace("secret ", "")
# Generated token so that you don't have to run server each time when testing. Comment out if running server
token = f.readline().rstrip().replace("token ", "") 

authorize_url =("https://www.dropbox.com/oauth2/authorize?client_id=" + APP_KEY + 
      "&response_type=code&redirect_uri=http://localhost:8000")

In [ ]:
# # Server code (can skip for now)
webbrowser.open(authorize_url)

HOST, PORT = "localhost", 8000
data = "unecessary"

# Create a socket (SOCK_STREAM means a TCP socket)

received = None
while (received == None or (isinstance(received,str) and len(received) < 3) or
    (isinstance(received,bytes) and "code" not in received.decode("utf-8"))):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        # Connect to server and send data
        sock.connect((HOST, PORT))
        print(sock.getsockname())

        # Receive data from the server and shut down

#         print("Sent: {}".format(data))
        sock.sendall(bytes(data + "\n", "utf-8"))
        received = str(sock.recv(1024), "utf-8")
#         print("Received: {}".format(received[0:10]))
    time.sleep(2)
token = re.search("(?<=code=)(.*?)(?=[!\s])", received).group()

In [33]:
url_files = "https://api.dropboxapi.com/2/file_requests/list"

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json"
}

data = {"path":""}

url="https://api.dropboxapi.com/2/files/list_folder"

req = urllib.request.Request(url,json.dumps(data).encode(),headers)
response = urllib.request.urlopen(req)
# response.status

files = json.loads(response.read().decode("utf-8"))

In [35]:
class DropboxRequest:
    def __init__(self,token):
        self.token = token
        
    @staticmethod
    def make_request(url, headers, data=None):
        """ Makes a request to Dropbox's API
        Args: url, headers of call
        Returns: a dict object
        """
        req = urllib.request.Request(url,json.dumps(data).encode(),headers)
        response = urllib.request.urlopen(req)
        return json.loads(response.read().decode("utf-8"))
    
    def list_folder(self,path):
        """ Gets a list of all files in a folder
        Args: 
            path: string path to the folder to look in
        Returns: a dict containing information about that folder
        """
        url = "https://api.dropboxapi.com/2/files/list_folder"
        header = {"Authorization": "Bearer " + token,
                  "Content-Type": "application/json"}
        data = {"path": path}
        return DropboxRequest.make_request(url, headers,data=data)
        
        